In [1]:
!nvidia-smi

Tue Apr 14 14:26:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.116.00   Driver Version: 418.116.00   CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000004:04:00.0 Off |                    0 |
| N/A   35C    P0    36W / 300W |      0MiB / 16130MiB |      0%   E. Process |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000004:05:00.0 Off |                    0 |
| N/A   39C    P0    36W / 300W |      0MiB / 16130MiB |      0%   E. Process |
+-------------------------------+----------------------+----------------------+
|   2  T

In [2]:
import os
import sys
sys.path.append('..')

import numpy as np
import tensorflow as tf

In [3]:
tf.__version__

'2.1.0'

In [23]:
latest = tf.train.latest_checkpoint(
    '/gpfs/alpine/scratch/pstjohn/bie108/uniparc_checkpoints/multinode_test_profile.34760/')

epoch = int(re.findall('.(\d{3})-', os.path.basename(latest))[0])

In [24]:
epoch

2

In [19]:
os.path.basename(latest)

'weights.002-14.57'

In [18]:
import re

2

'weights.002-14.57'

In [ ]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpu_devices))
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [ ]:
from bert.model import create_albert_model
from bert.losses import ECE, masked_sparse_categorical_crossentropy
from bert.optimization import create_optimizer

opt = create_optimizer(1E-4, 500, 10000)

strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")

with strategy.scope():
    model = create_albert_model(model_dimension=768,
                                transformer_dimension=768 * 4,
                                num_attention_heads=768 // 64,
                                num_transformer_layers=12,
                                vocab_size=24,
                                dropout_rate=0.,
                                max_relative_position=128)
    
    model.compile(
        loss=masked_sparse_categorical_crossentropy,
        metrics=[ECE],
        optimizer=opt,
        experimental_run_tf_function=True)    

    
model.summary()

In [ ]:
import os
from bert.dataset import create_masked_input_dataset
data_path = '/gpfs/alpine/proj-shared/bie108/split_uniref100'

train_data_dir = os.path.join(data_path, 'train_uniref100_split')
train_data_files = [os.path.join(train_data_dir, f) for f in os.listdir(train_data_dir)]

valid_data_dir = os.path.join(data_path, 'dev_uniref50_split')
valid_data_files = [os.path.join(valid_data_dir, f) for f in os.listdir(valid_data_dir)]

with tf.device('/CPU:0'):
    training_data = create_masked_input_dataset(
        sequence_path=train_data_files,
        max_sequence_length=512,
        fix_sequence_length=True,
        batch_size=12)
    
    training_data = training_data.repeat().prefetch(tf.data.experimental.AUTOTUNE)    

    valid_data = create_masked_input_dataset(
        sequence_path=valid_data_files,
        max_sequence_length=512,
        batch_size=12)

    valid_data = valid_data.repeat().prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
print(next(iter(training_data)))

In [ ]:
model.fit(training_data, steps_per_epoch=50, epochs=5,
          verbose=1, validation_data=valid_data, validation_steps=10)

In [ ]:
390 / 8

In [ ]:
552 / 12

In [ ]:
masked, true = next(iter(training_data))
predictions = model.predict(masked)

In [ ]:
masked[:, 0]

In [ ]:
model.save('/scratch/pstjohn/uniparc_checkpoints/test.h5')